In [27]:
#NOTE to list function generator with rm.list_resources(), needed to add a file in /etc/udev/rules.d
#Named it 99-siglent.rules
#Contains only 
# SUBSYSTEMS=="usb", ATTRS{idVendor}=="f4ec", ATTRS{idProduct}=="1102", MODE="0666"

#That will allow the function generator to be detected on a fresh installation

import pyvisa
rm = pyvisa.ResourceManager('@py')
rm.list_resources()

('USB0::62700::4354::SDG2XCAD5R3372::0::INSTR',)

In [28]:
sdg = rm.open_resource('USB0::62700::4354::SDG2XCAD5R3372::0::INSTR')
#sdg = rm.open_resource(ADDRESS)
#print(sdg.query("*IDN?"))

In [31]:
sdg.write("C1:OUTP OFF")
sdg.write("C1:BSWV FRQ, 2000")
sdg.write("C1:BSWV AMP, 15")
sdg.write("C1:BSWV WVTP, SQUARE")
waveInfo = sdg.query("C1:BSWV?")

while 1 < 2:
    if waveInfo[0] != "C":
        waveInfo = sdg.query("C1:BSWV?")
    else:
        break

In [30]:
freqIndex = waveInfo.find("FRQ")
voltIndexStart = waveInfo.find("AMP")
voltIndexEnd = waveInfo.find("V,AMPVRMS")

if waveInfo[freqIndex + 4:freqIndex + 8] != "2000":
    print("WARNING: INCORRECT FREQUENCY, MUST BE 2 kHz")
    raise SystemExit
if float(waveInfo[voltIndexStart+4:voltIndexEnd]) > 20.0:
    sdg.write("C1:AMP 1")
    print("WARNING: VOLTAGE TOO HIGH. VOLTAGE SHOULD BE NO GREATER THAN 20 V")
    raise SystemExit